In [19]:
## BIBLIOTECAS
import requests
from bs4 import BeautifulSoup

#Manipulação de Dados
import pandas as pd
import numpy as np

import shutil
import os
import pyautogui

import unidecode #arrumar textos

#tempo
import time
from time import sleep

import re #regex

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.font_manager as fm #para alterar as fontes dos graficos
import seaborn as sns

import pyperclip

import statistics
import math

from scipy import stats

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


# Definindo o estilo de fonte e tema
fonte = fm.FontProperties(family="Calibri", style="italic")

sns.set_style('whitegrid') #cor da grade(fundo)

import pyperclip
from tableone import TableOne

#TESTE ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

import seaborn

from statsmodels.stats.stattools import durbin_watson #teste de autocorrelação

from scipy.stats import chi2_contingency

In [2]:
def tabela_cont(col1, col2):
    a = df.groupby([col1, col2]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(col1, col2, opcao):
    result = tabela_cont(col1, col2)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [3]:
inicio = time.time() #iniciando o cronometro

df = pd.read_excel("4 - Dados_Score_Ano_vs_Acreditacao_vs_Unidade.xlsx")
df_bruto = df.copy()
display(df.sample(5))

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

,Unidade,acreditacao,Acreditação,Acreditação2,Ano,Regional,Estado,quant,1. As pessoas se apoiam umas às outras nesta unidade,2. Temos pessoas suficientes para lidar com o volume de trabalho,...,Trabalho em equipe entre as unidades hospitalares,Trabalho em equipe no âmbito das unidades,Transferências internas e passagens de plantão,Aprendizado organizacional - melhoria contínua,Expectativas de promoção da segurança dos supervisores e gerentes,Percepção gerais sobre segurança,Respostas não punitivas aos erros,Staffing,passou_a_ser_acreditado,status_eterno
235,Quinta D'Or,1,Qmentum,Qmentum,2021,SE,RJ,1261,71.132376,32.589641,...,56.281691,68.709937,52.915947,80.713926,68.876742,65.998158,38.122669,44.160649,0,1.0
80,Central Leste,0,0,0,2022,SE,SP,523,74.710425,32.629559,...,66.006026,75.450061,53.718840,86.628279,78.586493,64.175652,45.928373,51.330652,0,0.0
115,Cárdio Pulmonar,1,JCI,JCI,2021,NE,BA,1137,83.716814,58.267020,...,76.109535,80.828010,69.726396,88.349897,82.754494,73.705884,54.201082,60.281830,0,1.0
250,Rio Barra,0,0,0,2021,SE,RJ,424,70.047170,33.096927,...,65.413977,72.624184,59.874464,80.524059,70.801380,58.482072,38.153900,46.730107,1,NaN
25,Assunção,1,ONA,ONA 3,2017,SE,SP,606,69.102990,28.119800,...,65.604794,71.709803,51.186974,82.936872,71.680543,61.645411,47.039009,51.744487,0,1.0


Tempo para rodar a simulação: 0.68


In [4]:
lista_scores = ['1. As pessoas se apoiam umas às outras nesta unidade',
                '2. Temos pessoas suficientes para lidar com o volume de trabalho',
                '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
                '4. Nesta unidade, as pessoas se tratam com respeito',
                '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
                '6. Estamos ativamente buscando melhorias para a segurança do paciente',
                '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
                '8. Os funcionários sentem que seus erros são usados contra eles',
                '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
                '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
                '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
                '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
                '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
                '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
                '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
                '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
                '17. Temos problemas com a segurança do paciente nesta unidade',
                '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
                '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
                '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
                '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
                '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
                '23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
                '24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
                '25. Somos informados sobre os erros que acontecem nesta unidade',
                '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
                '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer',
                '28. Os funcionários têm receio de perguntar quando algo não parece certo',
                '29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
                '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
                '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?',
                '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
                '33. As unidades hospitalares não se coordenam bem entre si.',
                '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
                '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
                '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
                '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
                '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
                '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
                '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
                '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes',
                '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

In [5]:
colunas_adicionais = ['Abertura de comunicação', 'Feedback e comunicação sobre erros', 'Frequência de eventos relatados',
                      'Apoio de gestão hospitalar para segurança do paciente', 'Trabalho em equipe entre as unidades hospitalares',
                      'Trabalho em equipe no âmbito das unidades', 'Transferências internas e passagens de plantão',
                      'Aprendizado organizacional - melhoria contínua', 'Expectativas de promoção da segurança dos supervisores e gerentes',
                      'Percepção gerais sobre segurança', 'Respostas não punitivas aos erros', 'Staffing']
for i in colunas_adicionais:
    lista_scores.append(i)
print(lista_scores)

['1. As pessoas se apoiam umas às outras nesta unidade', '2. Temos pessoas suficientes para lidar com o volume de trabalho', '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa', '4. Nesta unidade, as pessoas se tratam com respeito', '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes', '6. Estamos ativamente buscando melhorias para a segurança do paciente', '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente', '8. Os funcionários sentem que seus erros são usados contra eles', '9. Erros que ocorreram levaram a mudanças positivas nesta unidade', '10. É apenas por acaso que erros mais sérios não acontecem por aqui', '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam', '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema', '13. Quando fazemos mudanças para melhorar a segur

In [6]:
df.head()

,Unidade,acreditacao,Acreditação,Acreditação2,Ano,Regional,Estado,quant,1. As pessoas se apoiam umas às outras nesta unidade,2. Temos pessoas suficientes para lidar com o volume de trabalho,...,Trabalho em equipe entre as unidades hospitalares,Trabalho em equipe no âmbito das unidades,Transferências internas e passagens de plantão,Aprendizado organizacional - melhoria contínua,Expectativas de promoção da segurança dos supervisores e gerentes,Percepção gerais sobre segurança,Respostas não punitivas aos erros,Staffing,passou_a_ser_acreditado,status_eterno
0,5 de outubro,1,ONA,ONA 2,2022,N,PA,311,74.757282,39.285714,...,65.235337,78.558495,54.567658,84.395514,79.463485,54.983432,45.862890,50.733033,0,1.0
1,Aeroporto,0,0,0,2022,NE,BA,525,67.123288,34.169884,...,63.705400,67.253311,57.644177,81.140402,71.754445,57.893346,38.125857,49.909629,0,0.0
2,Aliança,1,Qmentum,Qmentum,2021,NE,BA,1082,84.235075,55.844156,...,75.040345,82.245970,70.848337,89.222243,81.377292,72.762106,51.200263,60.606978,0,1.0
3,Aliança,1,Qmentum,Qmentum,2022,NE,BA,1029,83.544304,44.455253,...,70.155982,80.032914,67.845966,87.264457,79.034422,69.017709,52.215075,54.638196,0,1.0
4,Alpha Med,0,0,0,2017,SE,SP,244,68.879668,57.024793,...,65.385481,74.889841,49.424389,81.369523,78.799535,54.987140,43.080669,64.222550,0,0.0


In [7]:
df['controle'] = np.nan

for coluna in lista_scores:
    df['controle'] = "Intermediário"
    df['controle'][df[coluna] > 74.5] = "Pontos Fortes"
    df['controle'][df[coluna] < 50.4] = "Pontos Fracos"
    df[coluna] = df['controle']
    
    display(df[coluna].value_counts().to_frame().sort_index())

,1. As pessoas se apoiam umas às outras nesta unidade
Intermediário,283
Pontos Fortes,62


,2. Temos pessoas suficientes para lidar com o volume de trabalho
Intermediário,73
Pontos Fracos,272


,"3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa"
Intermediário,53
Pontos Fortes,292


,"4. Nesta unidade, as pessoas se tratam com respeito"
Intermediário,23
Pontos Fortes,322


,5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes
Intermediário,239
Pontos Fortes,1
Pontos Fracos,105


...

,Aprendizado organizacional - melhoria contínua
Intermediário,45
Pontos Fortes,300


,Expectativas de promoção da segurança dos supervisores e gerentes
Intermediário,201
Pontos Fortes,144


,Percepção gerais sobre segurança
Intermediário,327
Pontos Fortes,8
Pontos Fracos,10


,Respostas não punitivas aos erros
Intermediário,14
Pontos Fracos,331


,Staffing
Intermediário,198
Pontos Fracos,147


,Unidade,acreditacao,Acreditação,Acreditação2,Ano,Regional,Estado,quant,1. As pessoas se apoiam umas às outras nesta unidade,2. Temos pessoas suficientes para lidar com o volume de trabalho,...,Trabalho em equipe no âmbito das unidades,Transferências internas e passagens de plantão,Aprendizado organizacional - melhoria contínua,Expectativas de promoção da segurança dos supervisores e gerentes,Percepção gerais sobre segurança,Respostas não punitivas aos erros,Staffing,passou_a_ser_acreditado,status_eterno,controle
0,5 de outubro,1,ONA,ONA 2,2022,N,PA,311,Pontos Fortes,Pontos Fracos,...,Pontos Fortes,Intermediário,Pontos Fortes,Pontos Fortes,Intermediário,Pontos Fracos,Intermediário,0,1.0,Intermediário
1,Aeroporto,0,0,0,2022,NE,BA,525,Intermediário,Pontos Fracos,...,Intermediário,Intermediário,Pontos Fortes,Intermediário,Intermediário,Pontos Fracos,Pontos Fracos,0,0.0,Pontos Fracos
2,Aliança,1,Qmentum,Qmentum,2021,NE,BA,1082,Pontos Fortes,Intermediário,...,Pontos Fortes,Intermediário,Pontos Fortes,Pontos Fortes,Intermediário,Intermediário,Intermediário,0,1.0,Intermediário
3,Aliança,1,Qmentum,Qmentum,2022,NE,BA,1029,Pontos Fortes,Pontos Fracos,...,Pontos Fortes,Intermediário,Pontos Fortes,Pontos Fortes,Intermediário,Intermediário,Intermediário,0,1.0,Intermediário
4,Alpha Med,0,0,0,2017,SE,SP,244,Intermediário,Intermediário,...,Pontos Fortes,Pontos Fracos,Pontos Fortes,Pontos Fortes,Intermediário,Pontos Fracos,Intermediário,0,0.0,Intermediário
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,Vivalle,1,JCI,JCI,2022,SE,SP,918,Intermediário,Pontos Fracos,...,Intermediário,Intermediário,Pontos Fortes,Intermediário,Intermediário,Pontos Fracos,Pontos Fracos,0,1.0,Pontos Fracos
341,Yutaka,1,ONA,ONA 3,2022,N,PA,296,Intermediário,Pontos Fracos,...,Pontos Fortes,Intermediário,Pontos Fortes,Pontos Fortes,Intermediário,Pontos Fracos,Intermediário,0,1.0,Intermediário
342,viValle,1,Qmentum,Qmentum,2015,SE,SP,382,Intermediário,Pontos Fracos,...,Intermediário,Pontos Fracos,Pontos Fortes,Intermediário,Intermediário,Pontos Fracos,Pontos Fracos,0,1.0,Pontos Fracos
343,viValle,1,Qmentum,Qmentum,2016,SE,SP,420,Intermediário,Pontos Fracos,...,Intermediário,Intermediário,Pontos Fortes,Intermediário,Intermediário,Pontos Fracos,Pontos Fracos,0,1.0,Pontos Fracos


In [17]:
for coluna in lista_scores:
    # Criar a tabela de cruzamento
    cross_table = pd.crosstab(df[coluna], df["acreditacao"])
    chi2, p, dof, expected = chi2_contingency(cross_table)
    print(p)
    display(tabela_cont(coluna, "acreditacao"))
    
    print('='*100)
    

1.0


acreditacao,0,1
1. As pessoas se apoiam umas às outras nesta unidade,,
Intermediário,54,229
Pontos Fortes,12,50


0.12857996863188872


acreditacao,0,1
2. Temos pessoas suficientes para lidar com o volume de trabalho,,
Intermediário,19,54
Pontos Fracos,47,225


0.16715269916844466


...

acreditacao,0,1
Percepção gerais sobre segurança,,
Intermediário,56,271
Pontos Fortes,6,2
Pontos Fracos,4,6


1.0


acreditacao,0,1
Respostas não punitivas aos erros,,
Intermediário,3,11
Pontos Fracos,63,268


0.31610651816331886


acreditacao,0,1
Staffing,,
Intermediário,42,156
Pontos Fracos,24,123


In [32]:
df.groupby(["Staffing", "acreditacao", "Ano"]).size().unstack()

Ano                        2014  2015  2016  2017  2018  2019  2020  2021  \
Staffing      acreditacao                                                   
Intermediário 0             NaN   3.0   2.0   5.0   5.0   6.0   9.0   5.0   
              1             1.0   8.0  16.0  19.0  16.0  17.0  24.0  25.0   
Pontos Fracos 0             2.0   NaN   1.0   1.0   3.0   3.0   1.0   7.0   
              1             9.0  13.0  10.0   9.0  11.0  14.0  13.0  19.0   

Ano                        2022  
Staffing      acreditacao        
Intermediário 0             7.0  
              1            30.0  
Pontos Fracos 0             6.0  
              1            25.0

In [36]:
df.groupby(["acreditacao", "Ano"])["Staffing"].count().unstack()

Ano,2014,2015,2016,2017,2018,2019,2020,2021,2022
acreditacao,,,,,,,,,
0,2,3,3,6,8,9,10,12,13
1,10,21,26,28,27,31,37,44,55


In [38]:
df["Staffing"]

0      Intermediário
1      Pontos Fracos
2      Intermediário
3      Intermediário
4      Intermediário
           ...      
340    Pontos Fracos
341    Intermediário
342    Pontos Fracos
343    Pontos Fracos
344    Pontos Fracos
Name: Staffing, Length: 345, dtype: object

NameError: name 'df' is not defined

<Figure size 1200x600 with 0 Axes>